In [1]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [2]:
%matplotlib inline
import utils
from utils import *
from __future__ import division, print_function

Using TensorFlow backend.


In [3]:
path = "data/weather/"
model_path = path + 'models/'
batch_size=1024

In [22]:
data = pd.read_csv(path + 'all_data.csv', index_col='DateUTC', parse_dates=True, usecols=['DateUTC','TemperatureF','DewpointF','PressureIn','WindSpeedMPH','WindSpeedGustMPH','Humidity','HourlyPrecipIn','dailyrainin','SolarRadiationWatts/m^2'])
data.index = data.index.tz_localize('UTC').tz_convert('US/Pacific')

data = data[data.TemperatureF > 0] # Remove bad temp
data = data[data.PressureIn   > 0] # Remove bad pressure
data = data[data.HourlyPrecipIn >= 0] # Remove bad rainfall

data = data.assign(forward_min = data.rolling('18h', closed='both').TemperatureF.min().shift(-18,'1h').asof(data.index))

data = data.assign(day_of_year = data.index.dayofyear, hour = data.index.hour)
data = data.assign(threshold = 'no')
data.loc[data.forward_min < 32,'threshold'] = '32'

data = data.merge(pd.get_dummies(data.threshold, prefix='threshold'), left_index=True, right_index=True)

data.head()

,TemperatureF,DewpointF,PressureIn,WindSpeedMPH,WindSpeedGustMPH,Humidity,HourlyPrecipIn,dailyrainin,SolarRadiationWatts/m^2,forward_min,day_of_year,hour,threshold,threshold_32,threshold_no
DateUTC,,,,,,,,,,,,,,,
2016-02-16 11:12:00-08:00,73.8,50.5,29.29,0.0,0.0,44,0.0,0.0,17.48,73.8,47,11,no,0,1
2016-02-16 11:14:00-08:00,74.5,50.5,29.29,0.0,0.0,43,0.0,0.0,17.48,73.8,47,11,no,0,1
2016-02-16 11:15:00-08:00,74.5,50.5,29.28,0.0,0.0,43,0.0,0.0,17.48,73.8,47,11,no,0,1
2016-02-16 11:16:00-08:00,74.5,50.5,29.27,0.0,0.0,43,0.0,0.0,17.53,73.8,47,11,no,0,1
2016-02-16 11:18:00-08:00,74.5,50.5,29.28,0.0,0.0,43,0.0,0.0,17.48,73.8,47,11,no,0,1


In [23]:
seq_length=5
fp_fn_ratio=5
len(data)

124028

In [24]:
import dask.delayed as ddelayed

In [73]:
from pandas.tseries.offsets import *

offsets = pd.Series(DateOffset(minutes = 30*(3**i-1)) for i in range(seq_length-1,-1,-1))
wanted_cols = data[:100][['TemperatureF','DewpointF','PressureIn','WindSpeedMPH','WindSpeedGustMPH','Humidity','HourlyPrecipIn','dailyrainin','SolarRadiationWatts/m^2','day_of_year', 'hour']]
indexes = data.index[:100]
dwanted_cols = dd.from_pandas(wanted_cols, chunksize=10)

all_y = data[:100][['threshold_32']]

In [99]:
all_x = []

def get_history(x):
    indices = indexes.asof(x - offsets)
    return dwanted_cols.loc[[i for i in indices]]

for idx, _ in dwanted_cols.iterrows():
    all_x.append(ddelayed(get_history)(idx))

all_x = dd.compute(all_x)

TypeError: Cannot convert input [0   2016-02-25 02:21:00-08:00
1   2016-02-26 05:21:00-08:00
2   2016-02-26 14:21:00-08:00
3   2016-02-26 17:21:00-08:00
4   2016-02-26 18:21:00-08:00
dtype: datetime64[ns, US/Pacific]] of type <class 'pandas.core.series.Series'> to Timestamp